In [ ]:
# %load DataReader.py
import xlrd
import json
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import datetime

class item(): #Our objects of items with their features

    def __init__(self, name, price_history, weapon_type, rarity, wear, stattrak):

        self.name = name
        start_date = datetime.date(2013, 8, 15)
        self.price_history = price_history['prices']
        for i in range(len(self.price_history)):
            date = self.price_history[i][0].split()
            
            month_list = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
            year = date[2]
            day = date[1]
            month = month_list.index(date[0]) + 1
            release_date = datetime.date(int(year), int(month), int(day))
            diff_date = release_date - start_date
            self.price_history[i][0] = diff_date.days
            
        
        self.weapon_type = weapon_type
        self.rarity = rarity
        self.wear = wear
        self.stattrak = stattrak
        self.data = [self.weapon_type, self.rarity, self.wear, self.stattrak]
        self.target = price_history


def makeItems():
    print("Making items...")
    checker = " " #checker will later be set to value in excel file
    item_array = [] #our array our objects
    wears = ["Battle-Scarred", "Well-Worn", "Field-Tested", "Minimal Wear", "Factory New"]
    y = 1
    while(checker != "NULL"):  #checks to see if we are at end of excel file
        if(str(sheet.cell_value(y, 0)) != "NULL"):
            wep_wear = wears[int((sheet.cell_value(y, 4)) - 1)]
            string = str((sheet.cell_value(y, 0)) + " - " + str(wep_wear)) 
            with open("data points/" + string + ".json") as data_file: #opens a json file that has the same name as the name attribute
                data = json.load(data_file) #loads the content of file, which contains an array of its pricing history
                #print(data)
            prices = data
            thing = item(sheet.cell_value(y, 0), prices, sheet.cell_value(y, 2), sheet.cell_value(y, 3), sheet.cell_value(y, 4), sheet.cell_value(y, 5)) #create an object with all of the dat from json and excel file
            item_array.append(thing) #append it to our object list
            
        y = y + 1 #increment and update our checker for the end of the file
        checker = sheet.cell_value(y, 0)
    print("Finished!")
    return item_array






file_location = "C:/Users/ForceM/AppData/Local/Programs/Python/Python35-32/item_prices/DataReader.xlsx" #opens a excel file with our data
print("Data Found!")
workbook = xlrd.open_workbook(file_location)
sheet = workbook.sheet_by_index(0)
items = []
items = makeItems()
print("Training...")
learning_rate = 0.001
num_iterations = 50


total_item = 0
for t in range(36):
    total_item += int(len(items[t].price_history))
    

total_test = 0
for t in range(12):
    total_test += int(len(items[t+36].price_history))
    
price_history = np.zeros((total_item, 1))
items_to_train = np.zeros((total_item, 5))
price_test = np.zeros((total_test, 1))
days_test = np.zeros((total_test, 1))
days_train = np.zeros((total_item, 1))
items_to_test = np.zeros((total_test, 5))

b = 0
total_price = total_test + total_item
nor_m = np.zeros((total_price, 1))
for u in range(48):
    for h in range(len(items[u].price_history)):
        nor_m[b] = ([items[u].price_history[h][1]])
        b += 1
        
#nor_m = (nor_m - nor_m.mean()) / (nor_m.max() - nor_m.min())
nor_m = (nor_m - nor_m.min()) / (nor_m.max() - nor_m.min())

n = 0
for i in range(36):
    for j in range(len(items[i].price_history)):
        items_to_train[n] = [items[i].data[0], items[i].data[1], items[i].data[2], items[i].data[3], items[i].price_history[j][0]]
        price_history[n] = nor_m[n]
        #price_history[n] = price_history[n] * 1000
        days_train[n] = [items[i].price_history[j][0]]
        n += 1
        
m = 0
for p in range(12):
    for k in range(len(items[p+36].price_history)):
        items_to_test[m] = [items[p+36].data[0], items[p+36].data[1], items[p+36].data[2], items[p+36].data[3], items[p+36].price_history[k][0]]
        price_test[m] = nor_m[m+total_item]
        #price_test[m] = price_test[m] * 1000
        days_test[m] = [items[p+36].price_history[k][0]]
        m += 1
        

        

items_X_train = np.matrix(items_to_train)
items_Y_train = price_history

#items_Y_train = (items_Y_train - items_Y_train.mean()) / (items_Y_train.max() - items_Y_train.min())

items_X_test = np.matrix(items_to_test)
items_Y_test = price_test

print("Testing...")




#regr = linear_model.LogisticRegression()
#regr.fit(items_X_train, items_Y_train.ravel())

regr = linear_model.LinearRegression()
regr.fit(items_X_train, items_Y_train)

items_Y_pred = regr.predict(items_X_test)

print("Error for Linear Regression: ", end="")
print(mean_squared_error(items_Y_test, items_Y_pred))



svr_rbf = SVR(kernel='rbf')
y_rbf = svr_rbf.fit(items_X_train, items_Y_train).predict(items_X_train)

svr_poly = SVR(kernel='poly')
y_poly = svr_poly.fit(items_X_train, items_Y_train).predict(items_X_train)


print("Done!")




#plt.scatter(items_Y_train, y_rbf, color='black')


plt.scatter(days_train, items_Y_train, color='orange')
plt.scatter(days_train, y_rbf, color='blue')
plt.scatter(days_train, y_poly, color='green')


#plt.xticks("Days on Market")
#plt.yticks("Price in US Dollar")

print(regr.coef_)
print(items_Y_test[1])
print(items_Y_pred[1])

#print(items_Y_test.shape)

plt.show()


In [ ]:
print(5)